In [1]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib as plt

%matplotlib inline

In [4]:
from sqlalchemy import create_engine

postgres_str = ('postgresql://localhost/crossvalidated')
cnx = create_engine(postgres_str)

In [5]:
pd.read_sql_query('''SELECT * FROM Posts LIMIT 5;''', cnx)

,id,posttypeid,acceptedanswerid,parentid,creationdate,score,viewcount,body,owneruserid,lasteditoruserid,...,lasteditdate,lastactivitydate,title,tags,answercount,commentcount,favoritecount,closeddate,communityowneddate,jsonfield
0,1,1,15.0,NaN,2010-07-19 19:12:12.510,42,3654.0,<p>How should I elicit prior distributions fro...,8,NaN,...,NaT,2018-12-29 18:42:01.680,Eliciting priors from experts,<bayesian><prior><elicitation>,5.0,1,29.0,None,NaT,None
1,2,1,59.0,NaN,2010-07-19 19:12:57.157,31,30036.0,<p>In many different statistical methods there...,24,NaN,...,2010-08-07 17:56:44.800,2017-11-22 12:15:07.030,What is normality?,<distributions><normality-assumption>,7.0,1,11.0,None,NaT,None
2,3,1,5.0,NaN,2010-07-19 19:13:28.577,69,6221.0,<p>What are some valuable Statistical Analysis...,18,183.0,...,2011-02-12 05:50:03.667,2013-05-27 14:48:36.927,What are some valuable Statistical Analysis op...,<software><open-source>,19.0,3,40.0,None,2010-07-19 19:13:28.577,None
3,4,1,135.0,NaN,2010-07-19 19:13:31.617,21,35577.0,<p>I have two groups of data. Each with a dif...,23,NaN,...,NaT,2010-09-08 03:00:19.690,Assessing the significance of differences in d...,<distributions><statistical-significance>,5.0,2,11.0,None,NaT,None
4,5,2,NaN,3.0,2010-07-19 19:14:43.050,88,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23,23.0,...,2010-07-19 19:21:15.063,2010-07-19 19:21:15.063,None,None,NaN,3,NaN,None,2010-07-19 19:14:43.050,None
